In [71]:
!which python

/home/shubham/venv/bin/python


In [18]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
# import torch
from typing import List
import re
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import RecognizerResult, OperatorConfig

from presidio_analyzer import AnalyzerEngine, PatternRecognizer, EntityRecognizer, Pattern, RecognizerResult
from presidio_analyzer.recognizer_registry import RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngine, SpacyNlpEngine, NlpArtifacts
from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer

In [3]:
titles_list = ["Sir", "Ma'am", "Madam", "Mr.", "Mrs.", "Ms.", "Miss", "Dr.", "Professor"]

In [58]:

adhar_pattern = Pattern(name="adhar_pattern",regex="\d{12}", score = 0.9)
adhar_recognizer = PatternRecognizer(supported_entity="ADHAR", patterns = [adhar_pattern])
titles_recognizer = PatternRecognizer(supported_entity="TITLE", deny_list=titles_list)

In [61]:
class NumbersRecognizer(EntityRecognizer):
    
    expected_confidence_level = 0.7 # expected confidence level for this recognizer
    
    def load(self) -> None:
        """No loading is required."""
        pass

    def analyze(
        self, text: str, entities: List[str], nlp_artifacts: NlpArtifacts
    ) -> List[RecognizerResult]:
        """
        Analyzes test to find tokens which represent numbers (either 123 or One Two Three).
        """
        results = []
        
        # iterate over the spaCy tokens, and call `token.like_num`
        for token in nlp_artifacts.tokens:
            if token.like_num:
                result = RecognizerResult(
                    entity_type="NUMBER",
                    start=token.idx,
                    end=token.idx + len(token),
                    score=self.expected_confidence_level
                )
                results.append(result)
        return results

In [62]:
new_numbers_recognizer = NumbersRecognizer(supported_entities=["NUMBER"])


In [63]:
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

In [64]:
analyzer.registry.add_recognizer(new_numbers_recognizer)
analyzer.registry.add_recognizer(adhar_recognizer)
analyzer.registry.add_recognizer(titles_recognizer)

In [74]:
x = ["a","b","c"]

In [75]:
x

['a', 'b', 'c']

In [79]:
x.remove("b")

In [80]:
x

['a']

In [73]:
print([
    'PERSON',
    'CREDIT_CARD',
    'EMAIL_ADDRESS',
    'IP_ADDRESS',
    'PHONE_NUMBER',
    'CREDIT_CARD',
    'CRYPTO',
#     'DATE_TIME',
#     'NRP',
    'LOCATION',
    "NUMBER",
    "ADHAR",
    "TITLE"
#     'URL'
])

['PERSON', 'CREDIT_CARD', 'EMAIL_ADDRESS', 'IP_ADDRESS', 'PHONE_NUMBER', 'CREDIT_CARD', 'CRYPTO', 'LOCATION', 'NUMBER', 'ADHAR', 'TITLE']


In [65]:
ENT_TYPES = [
    'PERSON',
    'CREDIT_CARD',
    'EMAIL_ADDRESS',
    'IP_ADDRESS',
    'PHONE_NUMBER',
    'CREDIT_CARD',
    'CRYPTO',
#     'DATE_TIME',
#     'NRP',
    'LOCATION',
    "NUMBER",
    "ADHAR",
    "TITLE"
#     'URL'
]

In [66]:
sentence = "hello my name is Mr. andy, i am calling from whitehat. My number is 8050043862, and live in bangalore indranagar,email_id andybubbi@tmail.com,adhar number 999999990019, one nine nine one two"

In [68]:
analyzer.analyze(sentence, language='en', entities=ENT_TYPES)

[type: TITLE, start: 17, end: 20, score: 1.0,
 type: EMAIL_ADDRESS, start: 122, end: 141, score: 1.0,
 type: ADHAR, start: 155, end: 167, score: 0.9,
 type: PERSON, start: 21, end: 25, score: 0.85,
 type: LOCATION, start: 92, end: 112, score: 0.85,
 type: PHONE_NUMBER, start: 68, end: 78, score: 0.75,
 type: NUMBER, start: 68, end: 78, score: 0.7,
 type: NUMBER, start: 155, end: 167, score: 0.7,
 type: NUMBER, start: 169, end: 172, score: 0.7,
 type: NUMBER, start: 173, end: 177, score: 0.7,
 type: NUMBER, start: 178, end: 182, score: 0.7,
 type: NUMBER, start: 183, end: 186, score: 0.7,
 type: NUMBER, start: 187, end: 190, score: 0.7]

In [69]:
ents = analyzer.analyze(sentence, language='en', entities=ENT_TYPES)
results = anonymizer.anonymize(sentence, analyzer_results=ents)

In [70]:
results.text

'hello my name is <TITLE> <PERSON>, i am calling from whitehat. My number is <PHONE_NUMBER>, and live in <LOCATION>,email_id <EMAIL_ADDRESS>,adhar number <ADHAR>, <NUMBER> <NUMBER> <NUMBER> <NUMBER> <NUMBER>'